# The Hodgkin-Huxley Model

This week in programming club we're going to implement the Hodgkin-Huxley model to simulate an action potential.  Let's begin by considering a single neuron.  Or rather, let's consider the axon of a neuron.  This axon contains two ion channels, the voltage-gated Na<sup>+</sup> channel and the voltage-gated K<sup>+</sup> channel.  This axon also has the nice property that its voltage is uniform throughout is directly proportional to the charge stored on the inside of its membrane.  In other words, our axon is a capacitor.  The equation for a capacitor is

$$C=Q/V$$

where C is capacitance, V is voltage, and Q is the charge stored in the membrane.  You can see that as the charge stored in the membrane doubles, the voltage also doubles. We can rearrange this equation to look like this: 

$$CV=Q$$

If we differentiate with respect to time, we get this: 

$$C \dfrac{dV}{dt}=I$$

where I is current, since current is simply the rate of change of charge over time.  This equation is also known as the 'single-compartment model'.  It is the basis of our model.  This equation indicates that we can know the rate of change of the voltage if we know the current passing through the ion channels in our axon.  If the capacitance is 1nF (capacitance won't change over time, so it is a constant in our model) and the current I=10 nA, then rate of change of voltage is 10 V/s. If our starting voltage was -50mV, then over 1 ms our voltage would increase by 10mV, and at the end of that millisecond would be at -40 mV.  

Where do Hodgkin and Huxley come into this?  We only have two classes of ion channels (Na<sup>+</sup> and K<sup>+</sup>), so the current flowing into our axon should be the sum of the currents flowing through these ion channels.  Once we know the currents, we can get the volgate, which is ultimately what we are interested in.  This is where Hodgkin and Huxley help us.

## The delayed-rectifier K<sup>+</sup> channel
According to Ohms law, when the voltage (relative to equilibrium potential $E$) across a membrane doubles, current across the membrane doubles.  Another (stronger) way of saying that is that voltage and current are linearly related:
$$(V-E)g=I$$

$E$ is the equilibrium potential for that channel.  $g$ stands for conductance, the constant of porportionality. You can think of the conductance as how easy it is for charged ions to get across the membrane.  Conductance goes up when the number of channels open goes up.  This relationship is also linear:  if 5 ion channels are open, the total conductance through the membrane is 5 times greater than if a single channel was open.  The total conductance for a single class of ion channels at any given time equals the maximum possible conductance for those ion channels times the probability that each one is open.  If the total conductance for all K<sup>+</sup> was 100 nS, and the probability of any given channel being open is .5, then the conductance would be 50 nS.  We started out wanting to know what the current flowing through the K<sup>+</sup> channels is.  Now the problem reduces to, what is the open probability of a channel at a given time?

Without going through the experimental details, [Hodgkin and Huxley](http://www.ncbi.nlm.nih.gov/pmc/articles/PMC1392413/) figured that out for us.  The delayed-rectifier K<sup>+</sup> channel is composed of four subunits, each one must undergo a conformational change if the channel is going to open.  Let's let n be the probability that a single subunit is in its open position.  Then the probability that all four subunits are in the open position is n<sup>4</sup>.  In equation form:

$$ P(K^{+}_{open}) = n^{4} $$

Now lets focus on that variable $n$ .  The subunit is going back and forth between being in its open and closed conformation.  Like any chemical reaction, the rate going from one state to another will be proportional to the starting state.  The rate going from the open state to the closed state will be porportional to $n$, and the rate going from the closed state to the open state will be proportional to $1-n$. Also, there is a voltage dependence for both the open and closing rates.  Let's call the voltage dependence of the opening rate $\alpha_n(V)$ and the closing rate $\beta_n(V)$.  Putting all this together, the overall rate of transition will be the sum of the forward and backward parts: 

$$ \dfrac{dn}{dt} = \alpha_n(V)(1-n) - \beta_n(V)n $$

We will use this equation to model how all the subunits open and close.  For the delayed-rectitifier K<sup>+</sup> channel, Hodgkin and Huxley found equations for the voltage dependencies:

$$ \alpha_n(V) = \dfrac{.01(V+55)}{1-e^{-.1(V+55)}} $$


$$ \beta_n(V) = 0.125e^{-0.0125(V+65)} $$



## The voltage gated Na<sup>+</sup> channel
Things only get slightly more complicated for the Na<sup>+</sup> channel, but all the logic we used with the K<sup>+</sup> channel applies.  We can model the Na<sup>+</sup> channel with four subunits also, and all of them must be open for the channel to be open.  The difference is, one of those subunits will have a different voltage dependency, so we need to treat it seperately.  Let's use $m$ to represent the probability that a normal subunit is open.  By 'normal', I mean that as the voltage increases, this subunit is more likely to be in its open conformation.  Let's use $h$ to represent the probability that the oddball subunit is open.  These two subunit types have opening and closing voltage dependent rates which Hodgkin and Huxley found, so let's write those out:

$$ \alpha_m(V) = \dfrac{.1(V+40)}{1-e^{-.1(V+40)}} $$

$$ \beta_m(V) = 4e^{-0.0556(V+65)} $$

$$ \alpha_h(V) = .07e^{-0.05(V+65)}  $$

$$ \beta_h(V) = \dfrac{1}{1+e^{-.1(V+35)}} $$

Similar to the K<sup>+</sup> channel, the probability that the channel is open is the product of the probabilities that all the subunits are in their open conformational state:

$$ P(Na^{+}_{open}) = m^{3}h $$

## Putting it all together

As I stated earlier, the total current through the membrane is the sum of all the individual components.  We've mentioned two kinds of ion channels, but let's add one more to make our model complete.  An axon at rest has some ion channels open.  These are permeable mostly to K<sup>+</sup>.  There is more K<sup>+</sup> inside most mammalian cells than outside of them, so if a K<sup>+</sup> permeable channel opens, they will diffuse down their concentration gradient and out of the cell. This makes the cell more negative.   This is why at rest, most cells have a negative voltage.  The resting conductance (mostly K<sup>+</sup>) is called the leak conductance, $g_L$.  Now we can write equation for the total current through the cell.

$$I=g_L(V-E_L)+g_K n^4(V-E_K) + g_{Na} m^3h(V-E_{Na})   $$

Now we have all the equations we need to construct our model.  In our model, we will have a starting voltage.  We will use that volgate to figure out what state all our channels are in, and use that to get the current.  When we have the current, we can get the change in voltage, and add that $\Delta V$ to the voltage from our previous step.  

In more detail, the algorithm goes like this: 
1. Initialize a starting voltage, and states for all the subunits.
2. Using that starting voltage, determine what the opening and closing rates of all the subunits is (All those $\alpha$s and $\beta$s).
3. Use those subunits' rates to determine the open probability of each ion channel.
4. Use the open probabilities to determine the currents flowing through them into the cell.
5. Sum up these currents and calculate how much the voltage will change

We will perform all those steps for every time step, using the voltage from the previous time step to calculate the voltage for the next.  It sounds like a lot of work, but in our model, most of that work will be performed on the computer for you!! You just have to input the equations! Sound easy? 

## Programming Club Activity
This week in programming club, we're going to break up into groups and see who can simulate an axon firing an action potential using these equations.  The groups will compete to solve each step.  Once one group comes up with a solution, they'll explain how they solved it to all the other groups.  Then we'll move onto the next step.  By the time we're done, everyone will have programmed the complete Hodgkin-Huxley model.  We'll give you the steps as you go.  